# Analyzing Topical Model of Facebook's Topics aiming towards Users. 

In [1]:
from __future__ import absolute_import, division, print_function
# for word vec
import gensim
from gensim.models import Word2Vec as w2v
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from sklearn.feature_extraction.text import TfidfVectorizer

import codecs
import glob
import multiprocessing
import os
import nltk
import pprint
import re
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import itertools

In [8]:
# split into words
def get_words(raw):
    """
    :raw: text that has not been cleaned yet
    output: Stemmed Tokens
    """
    from nltk.tokenize import word_tokenize
    import string
    from nltk.corpus import stopwords
    from nltk.stem.porter import PorterStemmer
    tokens = word_tokenize(raw)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuation from each word
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    porter = PorterStemmer()
    stemmed = [porter.stem(word) for word in words]
    return stemmed

def get_topic_distribution(lda_model, raw_input, dictionary):
    """Return a vecor of topical distribution of each document or text. 
    :param lda_model: the output of the function gensim.models.ldamodel.LdaModel
    :param raw_imput: raw chinese policy text or doc
    :param dictionary: the output of corpora.Dictionary() function which is the vocab.
    """
    from pandas import DataFrame
    other_texts = [ # needs tokenized
        get_words(raw_input)
    ]
    #dictionary = Dictionary(sentences)
    other_corpus = [dictionary.doc2bow(text) for text in other_texts]
    unseen_doc = other_corpus[0]
    vector = lda_model[unseen_doc][0]
    return(DataFrame.from_records(vector)[1])

# Does politcal advertisements have an impact on the US November election? 
## Testing on random sample of Facebook users:
- Only English 
- From October to December
- Testing the US election in November vs FB ads political influence

In [3]:
import random
df = pd.read_csv('../ASUS/desktop/Thesis/codes/fb/electiondata_topics.csv')
random.seed(10)
RANDOM_SAMPLE = df.iloc[:, 0:22].sample(10000) # just taking a random sample of the population

C:\Users\ASUS\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
RANDOM_SAMPLE['publicationTime'] = pd.to_datetime(RANDOM_SAMPLE['publicationTime']).dt.date
RANDOM_SAMPLE = RANDOM_SAMPLE.sort_values(by='publicationTime')
RANDOM_SAMPLE.head()

,ANGRY,HAHA,LIKE,LOVE,SAD,WOW,displaySource,fblinktype,id,images.count,...,nature,permaLink,postId,publicationTime,source,sourceLink,timeline,user,concatenatedText,concatLanguage
584,NaN,NaN,17.0,NaN,6.0,22.0,Cambridgeshire Live,posts,87e7bbdea3ccc6d29ea4e1b2989d8ab4cc22c557,1,...,organic,/cambridgeshirelive/posts/10155999466698031,1.0156e+16,2018-10-01,Cambridgeshire Live,https://www.facebook.com/cambridgeshirelive/,caramel-grapefruit-okra,olive-plantains-samosa,We're bringing you the latest on this. Live: 4...,en
937,NaN,NaN,1.0,1.0,NaN,NaN,Gianpiero Luciano,posts,39ddb136708376b94e1b72eb0d1e66541db0de17,1,...,sponsored,story_fbid=2153112908290759&id=100007762113982,2.15311e+15,2018-10-01,Gianpiero Luciano,https://www.facebook.com/profile.php,pasta-pepper-waffle,bread-watermelon-sundae,https://vimeo.com/19077323 ESISTERE band: GNUT...,en
627,NaN,NaN,NaN,NaN,NaN,NaN,Lorenzo Ivaldi,posts,d46de8ddb99d96cc89298d4711034057bc601171,1,...,sponsored,/lorenzo.ivaldi/posts/10217248020743047,1.02172e+16,2018-10-01,Lorenzo Ivaldi,https://www.facebook.com/lorenzo.ivaldi,kebab-cheese-succotash,honey-cereal-taco,https://m.youtube.com/watch?v=NU9JoFKlaZ0# Gre...,en
1290,NaN,NaN,2.0,NaN,NaN,NaN,London Wobblies,posts,9e055dbbc9d2931f3264c3dc654de2ffa96a0e7c,2,...,organic,/london.iww/posts/1827048114068920,1.82705e+15,2018-10-01,London Wobblies,https://www.facebook.com/london.iww/,spaghetti-hamburger-icecream,saffron-berry-garbanzo,::: Coordinated strike action spreads across t...,en
3224,NaN,NaN,10.0,NaN,NaN,NaN,Montreal.AI,photo,8595f15b2bca015f63196eeb3e12e854d49d1969,1,...,organic,/MontrealAI/photos/a.373023919709539/728014010...,7.28014e+14,2018-10-01,Montreal.AI,https://www.facebook.com/MontrealAI/,waffle-nachos-cucumber,sherbet-kohlrabi-bread,tfjs-vis A set of utilities for in browser vi...,en


In [6]:
# break into tokens for each sentence
import time
start = time.time()
sent = RANDOM_SAMPLE.concatenatedText

sentences = []
for i in sent:
    sentences.append(get_words(i))
end = time.time()
print(end-start, 'seconds')

44.952433586120605 seconds


In [9]:
import pyLDAvis.sklearn
import gensim
import os
import dill
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
from gensim.models import Word2Vec as w2v
import time

# Create Dictionary
start = time.time()
id2word = corpora.Dictionary(sentences)
dictionary = Dictionary(sentences)
# Create Corpus
texts = sentences

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True,
                                           minimum_probability=0.0)



pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

#pyLDAvis.display(vis)

C:\Users\ASUS\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [ ]:
pyLDAvis.show(vis)


Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]


127.0.0.1 - - [28/Apr/2019 12:05:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2019 12:06:57] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2019 12:06:57] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2019 12:06:57] "GET /LDAvis.js HTTP/1.1" 200 -


In [10]:
lda_model.print_topics()

[(0,
  '0.007*"trump" + 0.007*"live" + 0.005*"music" + 0.005*"us" + 0.004*"govern" + 0.004*"man" + 0.004*"say" + 0.004*"state" + 0.004*"said" + 0.004*"artist"'),
 (1,
  '0.009*"new" + 0.009*"get" + 0.009*"http" + 0.009*"year" + 0.007*"one" + 0.007*"time" + 0.007*"see" + 0.006*"like" + 0.005*"day" + 0.005*"nt"'),
 (2,
  '0.013*"spillet" + 0.008*"ticket" + 0.008*"de" + 0.007*"la" + 0.005*"http" + 0.005*"e" + 0.004*"og" + 0.004*"lydbølger" + 0.004*"berlin" + 0.003*"tour"')]

## Merge the original dataset with the Topical Distribution spread 

In [11]:
import time
start = time.time()
buckets = []
for i in RANDOM_SAMPLE.concatenatedText:
    buckets.append(get_topic_distribution(lda_model,i,dictionary))
groups= []
for i in range(0,len(buckets)):
    groups.append(pd.Series.to_frame((buckets[i])).T)

groups = pd.concat(groups).reset_index()
DATA =  pd.concat([RANDOM_SAMPLE.reset_index(drop=True), groups],axis=1).reset_index(drop=True)
end = time.time()
print(end-start)
import dill
dill.dump_session('FBenv.db')
#dill.load_session('FBenv.db')

100.28394412994385


In [12]:
DATA.tail()

,ANGRY,HAHA,LIKE,LOVE,SAD,WOW,displaySource,fblinktype,id,images.count,...,source,sourceLink,timeline,user,concatenatedText,concatLanguage,index,0,1,2
9995,NaN,NaN,27.0,NaN,NaN,1.0,Chuck DeVore,photo,9b633baa372ce7eda99f319ad9299a4671d27f06,0,...,Chuck DeVore,https://www.facebook.com/chuck.devore,corn-tea-grapefruit,churros-collards-crisp,"On Fox News' Fox & Friends this A.M., noted ho...",en,1,0.552779,0.440490,0.006731
9996,3.0,NaN,1.0,NaN,NaN,NaN,East Coast Foods Pub and Grill,posts,ef661c58808ce01ffbebfb634775f64adaffc2e7,2,...,East Coast Foods Pub and Grill,https://www.facebook.com/EastCoastFoods/,shawarma-cucumber-blueberry,churros-collards-crisp,"Do you know me? I went to East Coast, ordered ...",en,1,0.162762,0.826893,0.010345
9997,NaN,NaN,38.0,NaN,NaN,NaN,Vancouver Whitecaps FC,videos,6ec8828e8b0ff6b550ee1fdfafc125234484fc3f,3,...,Vancouver Whitecaps FC,https://www.facebook.com/whitecapsfc/,succotash-okra-manicotti,shawarma-yolk-cheese,The feel-good moment you've been waiting for -...,en,1,0.052338,0.937305,0.010357
9998,NaN,NaN,2.0,1.0,NaN,NaN,Rebecca Gibson,posts,abcc7a7678bbd53cc62ced4280649f7664cdf0c9,1,...,Rebecca Gibson,https://www.facebook.com/rebecca.gibson.1293,scallion-muffin-cake,gyros-waffle-milkshake,So I still have some things to say about Pipki...,en,1,0.257620,0.741866,0.000515
9999,NaN,79.0,55.0,NaN,NaN,43.0,io9,posts,14464f4c7758d6acd3ad93dd76aabe265783cbb2,1,...,io9,https://www.facebook.com/io9.com/,leek-wheatberry-pomelo,currant-melon-sandwich,The former Smallville star defends herself fro...,en,1,0.544947,0.430394,0.024659


In [13]:
DATA.columns = [           'ANGRY',             'HAHA',             'LIKE',
                   'LOVE',              'SAD',              'WOW',
          'displaySource',       'fblinktype',               'id',
           'images.count',  'impressionOrder',   'impressionTime',
                 'nature',        'permaLink',           'postId',
        'publicationTime',           'source',       'sourceLink',
               'timeline',             'user', 'concatenatedText',
         'concatLanguage',            'index',                  'Politics',
                        'Advertisement',                  'Other']

In [14]:
DATA.tail()

,ANGRY,HAHA,LIKE,LOVE,SAD,WOW,displaySource,fblinktype,id,images.count,...,source,sourceLink,timeline,user,concatenatedText,concatLanguage,index,Politics,Advertisement,Other
9995,NaN,NaN,27.0,NaN,NaN,1.0,Chuck DeVore,photo,9b633baa372ce7eda99f319ad9299a4671d27f06,0,...,Chuck DeVore,https://www.facebook.com/chuck.devore,corn-tea-grapefruit,churros-collards-crisp,"On Fox News' Fox & Friends this A.M., noted ho...",en,1,0.552779,0.440490,0.006731
9996,3.0,NaN,1.0,NaN,NaN,NaN,East Coast Foods Pub and Grill,posts,ef661c58808ce01ffbebfb634775f64adaffc2e7,2,...,East Coast Foods Pub and Grill,https://www.facebook.com/EastCoastFoods/,shawarma-cucumber-blueberry,churros-collards-crisp,"Do you know me? I went to East Coast, ordered ...",en,1,0.162762,0.826893,0.010345
9997,NaN,NaN,38.0,NaN,NaN,NaN,Vancouver Whitecaps FC,videos,6ec8828e8b0ff6b550ee1fdfafc125234484fc3f,3,...,Vancouver Whitecaps FC,https://www.facebook.com/whitecapsfc/,succotash-okra-manicotti,shawarma-yolk-cheese,The feel-good moment you've been waiting for -...,en,1,0.052338,0.937305,0.010357
9998,NaN,NaN,2.0,1.0,NaN,NaN,Rebecca Gibson,posts,abcc7a7678bbd53cc62ced4280649f7664cdf0c9,1,...,Rebecca Gibson,https://www.facebook.com/rebecca.gibson.1293,scallion-muffin-cake,gyros-waffle-milkshake,So I still have some things to say about Pipki...,en,1,0.257620,0.741866,0.000515
9999,NaN,79.0,55.0,NaN,NaN,43.0,io9,posts,14464f4c7758d6acd3ad93dd76aabe265783cbb2,1,...,io9,https://www.facebook.com/io9.com/,leek-wheatberry-pomelo,currant-melon-sandwich,The former Smallville star defends herself fro...,en,1,0.544947,0.430394,0.024659


In [15]:
ORGANIC = DATA[DATA.nature == 'organic']
SPONSOR = DATA[DATA.nature == 'sponsored']

In [16]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import pandas as pd
import math
import math
from bubbly.bubbly import add_slider_steps
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
from plotly import tools
import plotly.figure_factory as ff 
import warnings
warnings.filterwarnings('ignore')
import os

#ORGANIC['publicationTime']=pd.to_datetime(ORGANIC['publicationTime']).dt.date
EX = pd.DataFrame(ORGANIC.groupby('publicationTime')[['Politics','Advertisement','Other']].mean()).reset_index(inplace=False)

x= EX.publicationTime

trace1 = dict(
    x= x,
    y= EX['Advertisement'],
    name='Advertisement',
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5),
              #color='rgb(131, 90, 241)'),
    stackgroup='one'
)
trace2 = dict(
    x= x,
    y= EX['Politics'],
    name='Politics',
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5),
              #color='rgb(111, 231, 219)'),
    stackgroup='one'
)
trace3 = dict(
    x= x,
    y= EX['Other'],
    name='Other',
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5),
              #color='rgb(184, 247, 212)'),
    stackgroup='one')


data = [trace1, trace2, trace3]

layout = go.Layout(
    
    legend=dict(x=0, y=-.13,
       orientation="h")
)



C:\Users\ASUS\Anaconda3\lib\site-packages\ipywidgets\widgets\widget.py:281: DeprecationWarning:

Widget registration using a string name has been deprecated. Widget registration now uses a plain `@register` decorator.

C:\Users\ASUS\Anaconda3\lib\site-packages\plotly\grid_objs\grid_objs.py:8: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working



# Topical Distribution Spread from Organic Posts Before and After teh Election

In [17]:
fig = go.Figure(data=data, layout=layout)
iplot(fig) 

In [19]:

EX = pd.DataFrame(SPONSOR.groupby('publicationTime')[['Politics','Advertisement','Other']].mean()).reset_index(inplace=False)

x= EX.publicationTime

trace1 = dict(
    x= x,
    y= EX['Advertisement'],
    name='Sales Ad',
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5),
              #color='rgb(131, 90, 241)'),
    stackgroup='one'
)
trace2 = dict(
    x= x,
    y= EX['Politics'],
    name='Political Ad',
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5),
              #color='rgb(111, 231, 219)'),
    stackgroup='one'
)
trace3 = dict(
    x= x,
    y= EX['Other'],
    name='Other Ad',
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5),
              #color='rgb(184, 247, 212)'),
    stackgroup='one')


data = [trace1, trace2, trace3]

layout = go.Layout(
    
    legend=dict(x=0, y=-.13,
       orientation="h")
)



# Sponsored Posts Topical Spread Before and After the Election

In [20]:
fig = go.Figure(data=data, layout=layout)
iplot(fig) 

# How much does each User's exposure to these Politcal, Advertisements? 
- Study case: pick user with ID "churros-collards-crisp" because he or she has the most impressions

In [23]:
df.user.value_counts() # We will now ananlyze the User's exposure to these Politcal, Advertisements

churros-collards-crisp         20267
currant-melon-sandwich         12671
honey-cereal-taco              12579
pumpkin-potato-pistachio       12257
melon-pimiento-shallot         11582
pudding-enchilada-thyme         9445
nectarine-cantaloupe-pizza      9127
relish-cupcake-rhubarb          8295
fennel-tangelo-kumquat          8207
sherbet-kohlrabi-bread          7895
truffles-tea-succotash          7533
lunch-souvlaki-citrus           6826
olive-plantains-samosa          5130
tortillas-souffle-ravioli       4558
wasabi-danish-nachos            3950
peach-date-thyme                3772
cherry-pimiento-cilantro        3441
milkshake-pecan-reuben          3404
okra-citrus-sauce               3383
melon-scallion-bun              3084
tacos-pecan-hamburger           2903
eggplant-butter-thyme           2715
grapefruit-pretzel-arrugula     2713
okra-ravioli-tapioca            2644
shawarma-yolk-cheese            2611
lasagna-falafel-chocolate       2483
cupcake-beet-tamale             2446
p

In [26]:
USER = df[df['user']=='churros-collards-crisp']
USER['publicationTime'] = pd.to_datetime(USER['publicationTime']).dt.date
USER = USER.sort_values(by='publicationTime')

sent = USER.concatenatedText
sentences = []
for i in sent:
    sentences.append(get_words(i))

id2word = corpora.Dictionary(sentences)
dictionary = Dictionary(sentences)
# Create Corpus
texts = sentences

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True,
                                           minimum_probability=0.0)
lda_model.print_topics()

[(0,
  '0.017*"book" + 0.015*"get" + 0.014*"market" + 0.013*"free" + 0.012*"busi" + 0.011*"http" + 0.009*"use" + 0.007*"want" + 0.006*"client" + 0.006*"time"'),
 (1,
  '0.010*"day" + 0.009*"nt" + 0.009*"one" + 0.008*"year" + 0.008*"go" + 0.008*"christma" + 0.007*"time" + 0.007*"get" + 0.007*"see" + 0.006*"peopl"'),
 (2,
  '0.015*"trump" + 0.010*"presid" + 0.009*"us" + 0.008*"govern" + 0.008*"wall" + 0.007*"border" + 0.007*"american" + 0.007*"restaur" + 0.007*"state" + 0.007*"report"')]

In [ ]:
buckets = []
for i in USER.concatenatedText:
    buckets.append(get_topic_distribution(lda_model,i,dictionary))
groups= []
for i in range(0,len(buckets)):
    groups.append(pd.Series.to_frame((buckets[i])).T)

groups = pd.concat(groups).reset_index()
USER_DATA =  pd.concat([USER.reset_index(drop=True), groups],axis=1).reset_index(drop=True)


In [31]:
USER_DATA.columns = [           'ANGRY',             'HAHA',             'LIKE',
                   'LOVE',              'SAD',              'WOW',
          'displaySource',       'fblinktype',               'id',
           'images.count',  'impressionOrder',   'impressionTime',
                 'nature',        'permaLink',           'postId',
        'publicationTime',           'source',       'sourceLink',
               'timeline',             'user', 'concatenatedText',
         'concatLanguage',             'time',          'topic_0',
                'topic_1',          'topic_2',          'topic_3',
                'topic_4',          'topic_5',          'topic_6',
                'topic_7',          'topic_8',          'topic_9',
               'topic_10',         'topic_11',         'topic_12',
               'topic_13',         'topic_14',         'topic_15',
               'topic_16',         'topic_17',         'topic_18',
               'topic_19',         'topic_20',         'topic_21',
               'topic_22',         'topic_23',         'topic_24',
               'topic_25',         'topic_26',         'topic_27',
               'topic_28',         'topic_29',         'topic_30',
               'topic_31',         'topic_32',         'topic_33',
               'topic_34',         'topic_35',         'topic_36',
               'topic_37',         'topic_38',         'topic_39',
               'topic_40',         'topic_41',         'topic_42',
               'topic_43',         'topic_44',         'topic_45',
               'topic_46',         'topic_47',         'topic_48',
               'topic_49',            'index',                  'Advertisement',
                        'Other',                  'Politics']
USER_ORGANIC = USER_DATA[USER_DATA.nature == 'organic']
USER_SPONSOR = USER_DATA[USER_DATA.nature == 'sponsored']

In [63]:
from bubbly.bubbly import add_slider_steps
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
from plotly import tools
import plotly.figure_factory as ff 
import warnings
warnings.filterwarnings('ignore')
import os
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot


EX = pd.DataFrame(USER_ORGANIC.groupby('publicationTime')[['Politics','Advertisement','Other']].mean()).reset_index(inplace=False)

x= EX.publicationTime

trace1 = dict(
    x= x,
    y= EX['Advertisement'],
    name='Organic Forwarded Ads',
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5),
              #color='rgb(131, 90, 241)'),
    stackgroup='one'
)
trace2 = dict(
    x= x,
    y= EX['Politics'],
    name='Organic Political Posts',
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5),
              #color='rgb(111, 231, 219)'),
    stackgroup='one'
)
trace3 = dict(
    x= x,
    y= EX['Other'],
    name='Other Organic Posts',
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5),
              #color='rgb(184, 247, 212)'),
    stackgroup='one')

trace_median0 =  go.Scatter(x=['2018-11-07', '2018-11-07'],
                            y=[0,1],
                            mode="lines",
                            legendgroup="a",
                            showlegend=True,
                            marker=dict(size=20,
                                       line=dict(width=1),
                                       color="black"
                                       ),
                            name="US Mid-term Election 2018",
                            visible=True
                            )

data_organic = [trace1, trace2, trace3, trace_median0]

layout = go.Layout(

    legend=dict(x=0, y=-.13,
       orientation="h")
)

In [54]:
EX = pd.DataFrame(USER_SPONSOR.groupby('publicationTime')[['Politics','Advertisement','Other']].mean()).reset_index(inplace=False)

x= EX.publicationTime

trace1 = dict(
    x= x,
    y= EX['Advertisement'],
    name='Sponsored Advertisement',
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5),
              #color='rgb(131, 90, 241)'),
    stackgroup='one'
)
trace2 = dict(
    x= x,
    y= EX['Politics'],
    name='Sponsored Politics',
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5),
              #color='rgb(111, 231, 219)'),
    stackgroup='one'
)
trace3 = dict(
    x= x,
    y= EX['Other'],
    name='Other Sponsors',
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5),
              #color='rgb(184, 247, 212)'),
    stackgroup='one')

trace_median1 =  go.Scatter(x=['2018-11-07', '2018-11-07'],
                            #y=[0,1],
                            mode="lines",
                            legendgroup="a",
                            showlegend=True,
                            marker=dict(size=20,
                                       line=dict(width=1),
                                       color="black"
                                       ),
                            name="US Mid-term Election 2018",
                            visible=True
                            )

data_sponsored = [trace1, trace2, trace3, trace_median1]

layout = go.Layout(
    yaxis=dict( autorange=True,
               fixedrange = False),
    legend=dict(x=0, y=-.13,
       orientation="h")
)

# How does Facebook impact A user's timeline impression during Political Elections?


## For USER ***churros-collards-crisp*** Personal timeline impression exposure (below)
> ## Organic Posts: political posts are about 10% higher in proportion before the midterm election on November 7th. 
> * We can conclude that he is more likely to be influenced by his own political social network. 

In [64]:
fig = go.Figure(data=data_organic, layout=layout)
iplot(fig) 

> ## Sponsored Posts for User **churros-collards-crisp**: 
> * Growth of 2% to 19% just 3 days before the US midterm election. 
> * He may be heavily influenced by Sponsored Political ads 2 weeks before the election.

In [55]:
fig = go.Figure(data=data_sponsored, layout=layout)
iplot(fig) 